# Descomposición del DOM

In [29]:
from bs4 import BeautifulSoup
from collections import Counter
from pathlib import Path

## Definiciones y utilidades estructurales

In [30]:
MAX_DEPTH = 12

IGNORED_TAGS = {
    "script", "style", "link", "meta", "svg", "noscript"
}

def normalize_classes(tag):
    return tuple(sorted(tag.get("class", [])))


def structural_fingerprint(tag, depth):
    if depth > MAX_DEPTH:
        return None
    if not tag.name or tag.name in IGNORED_TAGS:
        return None

    return (
        tag.name,
        normalize_classes(tag),
        depth
    )

def children_fingerprints(tag, depth):
    fps = []
    for child in tag.children:
        if getattr(child, "name", None):
            fp = structural_fingerprint(child, depth + 1)
            if fp:
                fps.append(fp)
    return Counter(fps)


In [31]:
def extract_structure(soup):
    structure = {}

    def walk(tag, depth=0):
        fp = structural_fingerprint(tag, depth)
        if not fp:
            return

        structure.setdefault(fp, []).append(
            children_fingerprints(tag, depth)
        )

        for child in tag.children:
            if getattr(child, "name", None):
                walk(child, depth + 1)

    walk(soup.body)
    return structure

In [32]:
def compare_structures(struct_a, struct_b):
    dynamic_candidates = []

    for key in set(struct_a) & set(struct_b):
        if struct_a[key] != struct_b[key]:
            dynamic_candidates.append(key)

    return dynamic_candidates

In [33]:
def load_html(path):
    return BeautifulSoup(
        Path(path).read_text(encoding="utf-8"),
        "html.parser"
    )

## extracción estructural

In [34]:
soup1 = load_html("data/raw/chrome.html")
soup2 = load_html("data/raw/firefox.html")

struct1 = extract_structure(soup1)
struct2 = extract_structure(soup2)

## Detección de zonas dinámicas (general)

In [35]:
dynamic_nodes = compare_structures(struct1, struct2)

#for tag, classes, depth in dynamic_nodes:
    #print(f"Tag: {tag}, Classes: {classes}, Depth: {depth}")

## Filtrado semántico de candidatos dinámicos

In [36]:
def is_relevant_candidate(tag, classes, depth):
    if not classes:
        return False
    if tag not in {"div", "section", "article"}:
        return False
    if depth < 2 or depth > 9:
        return False
    return True

In [37]:
print("Zonas dinámicas candidatas:\n")

for tag, classes, depth in dynamic_nodes:
    if is_relevant_candidate(tag, classes, depth):
        print(f"Tag: {tag}, Classes: {classes}, Depth: {depth}")

Zonas dinámicas candidatas:

Tag: div, Classes: ('e9EfHf',), Depth: 2
Tag: div, Classes: ('Tg0csd',), Depth: 3
Tag: div, Classes: ('Fgyi2e', 'caNvfd', 'rZj61'), Depth: 3
Tag: div, Classes: ('MjjYud',), Depth: 9
Tag: div, Classes: ('dURPMd',), Depth: 8


## Clasificación: contenedores dinámicos repetitivos dominantes

In [38]:
def item_fingerprint(tag):
    """
    Huella ligera para detectar items repetidos dentro de un contenedor
    """
    return (
        tag.name,
        tuple(sorted(tag.get("class", [])))
    )


In [39]:
def extract_child_patterns(tag):
    patterns = []

    for child in tag.children:
        if getattr(child, "name", None):
            if child.name in IGNORED_TAGS:
                continue
            patterns.append(item_fingerprint(child))

    return Counter(patterns)

In [40]:
def extract_repeated_patterns(
    child_counter,
    min_repetitions=2,
    coverage_threshold=0.6
):
    """
    Devuelve todos los patrones repetidos que,
    en conjunto, cubren la mayor parte del contenedor.
    """
    total = sum(child_counter.values())
    if total == 0:
        return {}

    repeated = {
        pattern: count
        for pattern, count in child_counter.items()
        if count >= min_repetitions
    }

    if not repeated:
        return {}

    coverage = sum(repeated.values()) / total

    if coverage >= coverage_threshold:
        return repeated

    return {}

In [41]:
def find_real_nodes(soup, target_fp):
    tag_name, classes, depth = target_fp
    results = []

    def walk(tag, current_depth=0):
        if current_depth > MAX_DEPTH:
            return

        if (
            tag.name == tag_name and
            tuple(sorted(tag.get("class", []))) == classes and
            current_depth == depth
        ):
            results.append(tag)

        for child in tag.children:
            if getattr(child, "name", None):
                walk(child, current_depth + 1)

    walk(soup.body)
    return results

### emisión de selectores CSS estructurales

In [42]:
def build_css_selector(tag, classes):
    if classes:
        return tag + "." + ".".join(classes)
    return tag


In [49]:
dynamic_containers = []

for fp in dynamic_nodes:
    tag, classes, depth = fp

    if not is_relevant_candidate(tag, classes, depth):
        continue

    nodes = find_real_nodes(soup1, fp)
    if not nodes:
        continue

    node = nodes[0]

    child_patterns = extract_child_patterns(node)
    repeated_patterns = extract_repeated_patterns(child_patterns)

    if not repeated_patterns:
        continue

    feed = {
        "container": build_css_selector(tag, classes),
        "items": []
    }

    for (item_tag, item_classes), count in repeated_patterns.items():
        feed["items"].append({
            "selector": "> " + build_css_selector(item_tag, item_classes),
            "count": count
        })

    dynamic_containers.append(feed)

dynamic_containers

[{'container': 'div.dURPMd',
  'items': [{'selector': '> div.MjjYud', 'count': 20}]}]

## Clasificación: contenedores dinámicos repetitivos compuesto

## Clasificación: contenedores dinámicos repetitivos Mixtos

## Zonas dinámicas no repetitivas

## Bloques únicos relevantes (estables)

# Motor de Inferencia

In [44]:
from dotenv import load_dotenv
import google.generativeai as genai
import zipfile
import os
import re

In [45]:
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash")

In [46]:
def generar_extension(prompt_principal, identificadores=None, codigo_referencia=None):
    """
    Genera el código de una extensión de Google Chrome en base a un prompt e información adicional.
    """
    
    mensaje = f""" Eres un **Generador Experto de Extensiones de Google Chrome (Manifest V3)**.

Tu objetivo es crear una extensión de Chrome **completamente funcional, segura y lista para cargar**, basándote rigurosamente en las instrucciones proporcionadas.

### Directrices Clave de Generación

1.  **Seguridad y Manifest V3 (CRUCIAL):**
    * **NO** se debe utilizar código JavaScript *inline* (ej. `<script>...</script>`) en archivos HTML (como `popup.html`). Todo JavaScript debe estar en archivos `.js` externos.
    * El archivo `manifest.json` debe estar configurado para **Manifest V3** y cumplir con las políticas de seguridad (CSP).
    * **NO** se debe hacer referencia a iconos, imágenes, fuentes o cualquier archivo externo que no esté estrictamente incluido en el código fuente de la respuesta.
2.  **Archivos:** Los archivos generados deben ser **autocontenidos** y **funcionales**.
3.  **Salida Estricta:** La respuesta debe contener **solamente** el código fuente de la extensión, **sin preámbulos, explicaciones, comentarios o texto adicional**.

---

### Parámetros de Extensión

**--- Instrucciones Principales (Función de la Extensión) ---**
{prompt_principal.strip()}

---

### Formato de Salida

Devuelve **únicamente** el código fuente, respetando **exactamente** el siguiente formato de bloques:

--- archivo: manifest.json ---
(contenido completo del archivo)
--- fin archivo ---

--- archivo: background.js ---
(contenido completo del archivo)
--- fin archivo ---

--- archivo: popup.html ---
(contenido completo del archivo, con el script JS enlazado externamente)
--- fin archivo ---

*(Incluye otros archivos, como `.js` o `.css`, solo si son necesarios para la funcionalidad, siguiendo el mismo formato de bloque.)*

--- archivo: popup.js ---
(contenido completo del archivo JS externo para popup.html)
--- fin archivo ---
. """

    respuesta = model.generate_content(mensaje)
    #generar_archivos_extension(respuesta.text)
    return respuesta.text

In [47]:
prompt = """Crea una extensión de Chrome que, al hacer clic en su icono, abra un popup que muestre una lista de las últimas 5 noticias tecnológicas de un sitio web popular. La extensión debe permitir al usuario actualizar la lista manualmente mediante un botón "Actualizar". Utiliza HTML, CSS y JavaScript para el diseño y la funcionalidad del popup. Asegúrate de que la extensión cumpla con las políticas de seguridad de Chrome y utilice Manifest V3."""

In [48]:
generar_extension(prompt)

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.